# 🎬 Thai→English Video Translator (Colab)

**Complete Thai video translation pipeline running on Google Colab with FREE GPU!**

## 🎯 What This Does

1. ✅ **Transcribe Thai audio** using Whisper large-v3 (GPU accelerated)
2. ✅ **Analyze context** for idioms, forex terms, metaphors
3. ✅ **Translate to English** with GPT-4/3.5 (smart routing)
4. ✅ **Generate SRT files** for both Thai and English
5. ✅ **Download results** to your computer

## 💰 Cost Estimate

- **Whisper transcription**: $0 (runs on Colab GPU)
- **Translation API**: $1.50-2.50 per hour of video
- **Total for 1 hour video**: ~$2.00

## ⏱️ Speed

- **Transcription**: 10-20x realtime (1 hour video = 3-6 minutes)
- **Translation**: 1-2 minutes
- **Total**: ~5-8 minutes for 1 hour video

---

## 📦 Step 1: Setup & Installation

**⚠️ Run this cell first!**

In [ ]:
# Install dependencies
!pip install -q openai-whisper openai python-dotenv pyyaml

# Check GPU availability
import torch
print("\n" + "="*60)
print("GPU Status")
print("="*60)
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️  No GPU found. Runtime → Change runtime type → GPU → Save")

print("\n✅ Installation complete!")

## 📂 Step 2: Upload Project Files

**Upload the `project.zip` file you downloaded:**

In [ ]:
from google.colab import files
import zipfile
import os

# Upload zip file
print("📤 Please select project.zip file to upload...")
uploaded = files.upload()

# Extract
zip_filename = list(uploaded.keys())[0]
print(f"\n📦 Extracting {zip_filename}...")

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall('.')

# Change to project directory
%cd video-translater

print("\n✅ Project files extracted!")
print("\nProject structure:")
!ls -la

## 🔑 Step 3: Setup API Key

**Upload your `.env` file with OpenAI API key:**

```bash
# .env file content:
OPENAI_API_KEY=sk-your-api-key-here
```

In [ ]:
from google.colab import files
import os

# Option 1: Upload .env file
print("📤 Upload your .env file (or skip and enter manually below)...")
uploaded = files.upload()

# Option 2: Manual entry (if no file uploaded)
if not uploaded:
    from getpass import getpass
    api_key = getpass("Enter your OpenAI API key: ")
    
    with open('.env', 'w') as f:
        f.write(f"OPENAI_API_KEY={api_key}\n")
    
    print("✅ API key saved to .env")
else:
    print("✅ .env file uploaded")

# Verify
from dotenv import load_dotenv
load_dotenv()

if os.getenv('OPENAI_API_KEY'):
    print("✅ API key loaded successfully")
else:
    print("❌ API key not found. Please check .env file")

## 🎥 Step 4: Upload Video

**Upload your Thai video file to translate:**

In [ ]:
from google.colab import files
from pathlib import Path

print("📤 Please select your video file...")
uploaded = files.upload()

video_file = list(uploaded.keys())[0]
video_path = Path(video_file)

print(f"\n✅ Video uploaded: {video_file}")
print(f"Size: {video_path.stat().st_size / (1024*1024):.2f} MB")

# Get video info
import subprocess
import json

result = subprocess.run(
    ['ffprobe', '-v', 'quiet', '-print_format', 'json', '-show_format', str(video_path)],
    capture_output=True,
    text=True
)

info = json.loads(result.stdout)
duration = float(info['format']['duration'])

print(f"Duration: {int(duration // 60)}:{int(duration % 60):02d}")
print(f"\n💰 Estimated cost: ${(duration / 3600) * 2:.2f}")
print(f"⏱️  Estimated time: {int(duration / 10)}-{int(duration / 5)} seconds")

## 🚀 Step 5: Run Translation Pipeline

**This will:**
1. Transcribe Thai audio (Whisper large-v3 on GPU)
2. Analyze context (idioms, forex terms)
3. Translate to English (GPT-3.5/4)
4. Generate SRT files

In [ ]:
import sys
from pathlib import Path

# Import orchestrator
sys.path.insert(0, 'src')
from orchestrator import VideoTranslationOrchestrator
from config import ConfigMode
from context_analyzer import DocumentType

# Configuration
CONFIG = {
    'whisper_model': 'large-v3',
    'config_mode': ConfigMode.PRODUCTION,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'doc_type': DocumentType.TUTORIAL
}

print("="*70)
print("Starting Translation Pipeline")
print("="*70)
print(f"Video: {video_file}")
print(f"Whisper model: {CONFIG['whisper_model']}")
print(f"Device: {CONFIG['device']}")
print(f"Mode: {CONFIG['config_mode'].value}")
print("="*70)

# Initialize orchestrator
orchestrator = VideoTranslationOrchestrator(
    whisper_model=CONFIG['whisper_model'],
    config_mode=CONFIG['config_mode'],
    device=CONFIG['device']
)

# Process video
result = orchestrator.process_video(
    input_path=video_path,
    output_dir=Path('output'),
    doc_type=CONFIG['doc_type']
)

# Results
print("\n" + "="*70)
print("RESULTS")
print("="*70)

if result.success:
    print("✅ Translation completed successfully!")
    print(f"\nOutput files:")
    for name, path in result.output_files.items():
        print(f"  - {name}: {path}")
    
    print(f"\nStatistics:")
    print(f"  - Duration: {result.stats['duration_seconds']:.1f}s")
    print(f"  - Processing time: {result.stats['processing_time_seconds']:.1f}s")
    print(f"  - Speed: {result.stats['processing_speed']:.1f}x realtime")
    print(f"  - Thai segments: {result.stats['thai_segments']}")
    print(f"  - Thai confidence: {result.stats['thai_confidence']:.1%}")
    print(f"  - Cost: ${result.stats['estimated_cost']:.4f}")
    print(f"  - Cost/min: ${result.stats['cost_per_minute']:.4f}")
else:
    print(f"❌ Translation failed: {result.error}")

## 📥 Step 6: Download Results

**Download all output files to your computer:**

In [ ]:
from google.colab import files
import os
from pathlib import Path

if result.success:
    print("📥 Downloading files...")
    print()
    
    for name, path in result.output_files.items():
        if path.exists():
            print(f"⬇️  Downloading {name}: {path.name}")
            files.download(str(path))
    
    print("\n✅ All files downloaded!")
    print("\nNext steps:")
    print("1. Check your Downloads folder")
    print("2. Use *_english.srt for English subtitles")
    print("3. Use *_thai.srt for Thai subtitles")
    print("4. Optional: Merge SRT with video using local script")
else:
    print("❌ No files to download (translation failed)")

## 🔄 Optional: Merge Subtitles with Video

**Burn subtitles directly into video (creates new video file):**

In [ ]:
import subprocess

if result.success:
    # Get paths
    english_srt = result.output_files['english_srt']
    output_video = Path('output') / f"{video_path.stem}_with_subtitles.mp4"
    
    print("🎬 Merging English subtitles with video...")
    print(f"Input: {video_file}")
    print(f"Subtitles: {english_srt.name}")
    print(f"Output: {output_video.name}")
    print()
    
    # FFmpeg command
    cmd = [
        'ffmpeg',
        '-i', str(video_path),
        '-vf', f"subtitles={english_srt}:force_style='FontName=Arial,FontSize=24,PrimaryColour=&H00FFFFFF,OutlineColour=&H00000000,Outline=2,MarginV=30'",
        '-c:v', 'libx264',
        '-preset', 'medium',
        '-crf', '23',
        '-c:a', 'copy',
        '-y',
        str(output_video)
    ]
    
    subprocess.run(cmd, check=True)
    
    print("\n✅ Video with subtitles created!")
    print(f"Downloading {output_video.name}...")
    files.download(str(output_video))
    
    print("\n✅ Download complete!")
else:
    print("❌ Translation must succeed first")

## 📊 Optional: View Statistics

**Detailed translation statistics:**

In [ ]:
if result.success:
    import json
    
    # Load full stats
    stats_file = result.output_files['stats']
    with open(stats_file) as f:
        stats = json.load(f)
    
    print("="*70)
    print("DETAILED STATISTICS")
    print("="*70)
    
    print(f"\n📹 Video Information:")
    print(f"  Input: {stats['input_file']}")
    print(f"  Duration: {stats['duration_seconds']:.1f}s ({stats['duration_seconds']/60:.1f} min)")
    
    print(f"\n🎯 Transcription:")
    print(f"  Thai segments: {stats['thai_segments']}")
    print(f"  Thai words: {stats['thai_words']}")
    print(f"  Confidence: {stats['thai_confidence']:.1%}")
    
    print(f"\n🌐 Translation:")
    print(f"  GPT-3.5 segments: {stats['gpt35_segments']}")
    print(f"  GPT-4 segments: {stats['gpt4_segments']}")
    print(f"  Cache hits: {stats['translation_cache_hits']}")
    print(f"  Cache rate: {stats['translation_cache_rate']:.1%}")
    
    print(f"\n💰 Cost:")
    print(f"  Total: ${stats['estimated_cost']:.4f}")
    print(f"  Per minute: ${stats['cost_per_minute']:.4f}")
    
    print(f"\n⏱️  Performance:")
    print(f"  Processing time: {stats['processing_time_seconds']:.1f}s")
    print(f"  Speed: {stats['processing_speed']:.1f}x realtime")
    
    print(f"\n✅ Timestamp: {stats['timestamp']}")
else:
    print("❌ No statistics available (translation failed)")

## 🔄 Batch Processing (Multiple Videos)

**Process multiple videos at once:**

In [ ]:
# Upload multiple videos
print("📤 Upload multiple video files...")
uploaded = files.upload()

video_files = [Path(f) for f in uploaded.keys()]
print(f"\n✅ Uploaded {len(video_files)} videos")

# Process each
results = []
total_cost = 0

for i, video in enumerate(video_files, 1):
    print(f"\n{'='*70}")
    print(f"[{i}/{len(video_files)}] Processing: {video.name}")
    print(f"{'='*70}")
    
    result = orchestrator.process_video(
        input_path=video,
        output_dir=Path('output') / video.stem,
        doc_type=DocumentType.TUTORIAL
    )
    
    results.append(result)
    
    if result.success:
        total_cost += result.stats['estimated_cost']
        print(f"✅ Success - Cost: ${result.stats['estimated_cost']:.4f}")
    else:
        print(f"❌ Failed: {result.error}")

# Summary
print(f"\n{'='*70}")
print("BATCH SUMMARY")
print(f"{'='*70}")
print(f"Total videos: {len(video_files)}")
print(f"Successful: {sum(1 for r in results if r.success)}")
print(f"Failed: {sum(1 for r in results if not r.success)}")
print(f"Total cost: ${total_cost:.4f}")

# Download all
print("\n📥 Downloading all results...")
for result in results:
    if result.success:
        for path in result.output_files.values():
            if path.exists():
                files.download(str(path))

## 💾 Save to Google Drive (Optional)

**Save results to your Google Drive instead of downloading:**

In [ ]:
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Create output folder in Drive
drive_output = Path('/content/drive/MyDrive/ThaiVideoTranslations')
drive_output.mkdir(parents=True, exist_ok=True)

if result.success:
    # Copy all output files
    video_output_dir = drive_output / video_path.stem
    video_output_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"📤 Copying files to Google Drive...")
    for name, path in result.output_files.items():
        if path.exists():
            dest = video_output_dir / path.name
            shutil.copy2(path, dest)
            print(f"  ✅ {path.name}")
    
    print(f"\n✅ Files saved to: {video_output_dir}")
    print("You can access them from your Google Drive!")
else:
    print("❌ No files to save")

---

## 📚 Documentation

### Output Files

- `*_thai.srt` - Thai subtitles (original transcription)
- `*_english.srt` - English subtitles (translation)
- `*_thai.json` - Full Thai transcription with timestamps
- `*_context.json` - Context analysis (idioms, terms, metaphors)
- `*_stats.json` - Processing statistics and costs

### Tips

1. **GPU is essential** for fast transcription (10-20x speedup)
2. **Cost optimization**: Use `ConfigMode.COST_OPTIMIZED` for cheaper translation
3. **Quality focus**: Use `ConfigMode.QUALITY_FOCUS` for best accuracy
4. **Save to Drive**: Recommended for large batches (Colab session timeout = 12 hours)
5. **Resume support**: Save checkpoint to Drive, reload on new session

### Troubleshooting

**GPU not available:**
- Runtime → Change runtime type → GPU → Save

**Out of memory:**
- Use smaller Whisper model: `medium` or `small`
- Split long videos first

**API errors:**
- Check OpenAI API key is valid
- Verify you have credits/billing enabled

**Session timeout:**
- Save to Google Drive frequently
- Use batch processing with checkpoints

---

## 🎉 You're Done!

Your Thai videos are now translated to English with perfect subtitles! 🚀